In [198]:
import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt
import requests
import json

In [199]:
with open('C:/Users/clare/Documents/Flatiron/PatternRecommender/.secrets/creds.json') as f:
    creds = json.load(f)

In [200]:
input_df = pd.read_csv('saved_100000_calls.csv')
input_df

,user,completed,rating,status,pattern_id,average_rating,rating_count
0,Strickpingu,2016/01/01,4.0,Finished,254399,4.606292,1049.0
1,Strickpingu,2015/04/23,4.0,Finished,309410,4.750000,44.0
2,Strickpingu,2012/11/04,4.0,Finished,119656,4.455452,1504.0
3,Strickpingu,2012/07/18,4.0,Finished,299579,4.573913,115.0
4,Strickpingu,NaN,3.0,In progress,629767,4.822222,45.0
...,...,...,...,...,...,...,...
154936,rogueriverfiber,NaN,NaN,In progress,808645,4.715596,109.0
154937,rogueriverfiber,NaN,NaN,Finished,525712,4.282051,39.0
154938,rogueriverfiber,NaN,NaN,NaN,875144,4.575342,73.0
154939,cookbaby,NaN,NaN,NaN,39772,4.233333,30.0


In [201]:
df_drop_nans = input_df[['user', 'pattern_id', 'rating']].dropna(subset = ['rating'])

df_drop_nans

,user,pattern_id,rating
0,Strickpingu,254399,4.0
1,Strickpingu,309410,4.0
2,Strickpingu,119656,4.0
3,Strickpingu,299579,4.0
4,Strickpingu,629767,3.0
...,...,...,...
154928,rogueriverfiber,721035,4.0
154932,rogueriverfiber,391215,4.0
154933,rogueriverfiber,578730,3.0
154934,rogueriverfiber,216488,4.0


In [202]:
df_drop_nans.describe()

,pattern_id,rating
count,1.027860e+05,102786.000000
mean,3.364344e+05,3.565233
std,3.037349e+05,0.702694
min,1.000000e+01,0.000000
25%,8.235400e+04,3.000000
50%,2.448235e+05,4.000000
75%,5.459900e+05,4.000000
max,1.158690e+06,5.000000


In [170]:
df_replace_nans = input_df[['user', 'pattern_id', 'rating', 'average_rating']]
rating_replace_nans = df_replace_nans['rating'].fillna(df_replace_nans['average_rating'])
df_replace_nans['rating'] = rating_replace_nans
df_replace_nans.drop(columns = 'average_rating', inplace = True)

df_replace_nans

<ipython-input-170-c256ad64ffaa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_replace_nans['rating'] = rating_replace_nans
C:\Users\clare\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,user,pattern_id,rating
0,hannahcf,17468,4.0
1,hannahcf,629964,4.0
2,hannahcf,287992,4.0
3,hannahcf,475167,4.0
4,hannahcf,544863,4.0
...,...,...,...
11117,creativratte,223414,4.0
11118,creativratte,467793,4.0
11119,creativratte,211562,4.0
11120,creativratte,250525,4.0


In [203]:
from surprise import Reader, Dataset
reader = Reader()

data_drop = Dataset.load_from_df(df_drop_nans, reader)

In [204]:
from surprise.model_selection import train_test_split

drop_trainset, drop_testset = train_test_split(data_drop, test_size=0.25)


In [205]:
drop_trainset.global_mean

3.5633358844971394

In [206]:
from surprise import SVD, accuracy
algo = SVD(n_factors = 50, n_epochs = 45, lr_all = 0.004, reg_all = 0.2)
algo.fit(drop_trainset)

In [207]:
predictions = algo.test(drop_testset)

In [208]:
accuracy.rmse(predictions)

RMSE: 0.6297


0.6297292053383162

In [177]:
 algo.fit(drop_trainset)

In [209]:
from surprise.model_selection import GridSearchCV

In [210]:
param_grid = {'n_factors':[5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
              'n_epochs': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 
              'lr_all': [0.002, 0.003, 0.004, 0.005],
              'reg_all': [0.2, 0.3, 0.4, 0.5, 0.6]}

gs_model = GridSearchCV(SVD,
                        param_grid=param_grid,
                        n_jobs = -1,
                        joblib_verbose=5)

gs_model.fit(data_drop)

gs_model.best_params

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 858 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 2154 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 3018 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 3504 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 27.9min
[Parallel(n_jobs=-1)]: Done 4584 tasks      | e

{'rmse': {'n_factors': 40, 'n_epochs': 45, 'lr_all': 0.002, 'reg_all': 0.2},
 'mae': {'n_factors': 5, 'n_epochs': 35, 'lr_all': 0.005, 'reg_all': 0.2}}

In [211]:
from surprise import SVD, accuracy
algo = SVD(n_factors = 40, n_epochs = 45, lr_all = 0.002, reg_all = 0.2)
algo.fit(drop_trainset)

In [212]:
predictions = algo.test(drop_testset)

In [213]:
accuracy.rmse(predictions)

RMSE: 0.6278


0.6277843792807158

In [179]:
predictions

[Prediction(uid='Rozieq', iid=229671, r_ui=3.0, est=3.618153948468545, details={'was_impossible': False}),
 Prediction(uid='rjc537', iid=283299, r_ui=3.0, est=3.06480512976292, details={'was_impossible': False}),
 Prediction(uid='cindeknits', iid=908295, r_ui=4.0, est=3.6862650602809737, details={'was_impossible': False}),
 Prediction(uid='fargerik', iid=161817, r_ui=4.0, est=3.95768801916462, details={'was_impossible': False}),
 Prediction(uid='Senior67', iid=154873, r_ui=4.0, est=3.8103528091280903, details={'was_impossible': False}),
 Prediction(uid='lisaladd', iid=98434, r_ui=4.0, est=3.7390620124320213, details={'was_impossible': False}),
 Prediction(uid='wuepfi', iid=667913, r_ui=4.0, est=3.894337792458644, details={'was_impossible': False}),
 Prediction(uid='scruz82', iid=243676, r_ui=4.0, est=3.310967322441874, details={'was_impossible': False}),
 Prediction(uid='Ona', iid=532791, r_ui=4.0, est=3.719113814782432, details={'was_impossible': False}),
 Prediction(uid='revsmf', iid

In [180]:
predictions[0].r_ui

3.0

In [181]:
predictions_df = pd.DataFrame({"user": [prediction.uid for prediction in predictions],
                   "item": [prediction.iid for prediction in predictions],
                   "actual": [prediction.r_ui for prediction in predictions],
                   "estimated" :[prediction.est for prediction in predictions]})
    

In [182]:
predictions_df[predictions_df['user'] == 'Ona'].describe()

,item,actual,estimated
count,12.000000,12.000000,12.000000
mean,212622.333333,3.750000,3.738348
std,149854.701308,0.452267,0.047157
min,10760.000000,3.000000,3.648831
25%,103543.000000,3.750000,3.719114
50%,176497.000000,4.000000,3.719114
75%,331984.500000,4.000000,3.779042
max,532791.000000,4.000000,3.824224


In [183]:
predictions_df

,user,item,actual,estimated
0,Rozieq,229671,3.0,3.618154
1,rjc537,283299,3.0,3.064805
2,cindeknits,908295,4.0,3.686265
3,fargerik,161817,4.0,3.957688
4,Senior67,154873,4.0,3.810353
...,...,...,...,...
2194,Halias,17533,3.0,3.514645
2195,JuliaAngeline,9866,4.0,3.161346
2196,FirstHome,248188,4.0,3.721050
2197,rose54,124800,4.0,3.950666


In [184]:
len(input_df['pattern_id'].unique())

7213

In [185]:
patterns_list = list(input_df['pattern_id'].unique())

In [186]:
users_list = list(input_df['user'].unique())

In [187]:
users_list

['hannahcf',
 'Diliel',
 'cathycrafts',
 'beadmneiae',
 'bsuthersan',
 'mek17',
 'Marteing',
 'awhite22',
 'KAW',
 'kmakens',
 'annabritak',
 'ahaas57',
 'Ona',
 'bellybutton',
 'eparys',
 'Littlebitobear',
 'robsa',
 'Senior67',
 'Neak',
 'bambiegirl',
 'himani196',
 'synnoeveb',
 'mswazlib',
 'susanpenz',
 'KnittingYall',
 'gillc',
 'Eua',
 'cmalena',
 'GentlyUsedCoat',
 'Mandala',
 'bonniebake',
 'Anisha',
 'oldskoolknitz',
 'susanmitch',
 'daiva0823',
 'marymiller',
 'rbrock',
 'rose54',
 'chellebelle99504',
 'Chloe13',
 'valdelgiudice',
 'newday',
 'Ryan875',
 'mudeyes',
 'SenorSoleil',
 'NatashaR',
 'kircin',
 'dlhknitter',
 'jeanmag',
 'lisaladd',
 'ppaddy',
 'kjax1776',
 'heartsknit',
 'chrisdee',
 'ComeAlongPond26',
 'khain622',
 'mrsvickie',
 'emilybluestar',
 'lese1',
 'lusjesdesign',
 'kdugger13',
 'yarnut',
 'Sileaf',
 'Lienel',
 'Urieka',
 'Augusts',
 'amsauder',
 'Muffinstitches',
 'saucysue',
 'raintreeinwind',
 'tracey312',
 'soentwined',
 'austinmom',
 'origamibard',


In [188]:
input_df[input_df['user'] == "Ona"]

,user,completed,rating,status,pattern_id,average_rating,rating_count
182,Ona,2015/04/22,3.0,Finished,546649,4.615385,13.0
183,Ona,2015/04/12,3.0,Finished,393371,4.531915,47.0
184,Ona,2014/11/26,NaN,Finished,213483,4.757460,1307.0
185,Ona,2014/08/29,3.0,Finished,327637,4.630499,341.0
186,Ona,2013/03/20,4.0,Finished,359828,4.586093,302.0
187,Ona,2013/01/13,4.0,Finished,89104,3.933333,15.0
188,Ona,2013/01/01,4.0,Finished,134174,4.523490,298.0
189,Ona,2012/12/27,4.0,Finished,357468,4.333333,60.0
190,Ona,2012/12/07,4.0,Finished,10760,4.209974,381.0
191,Ona,2012/12/04,4.0,Finished,293353,4.626655,2341.0


In [159]:
def get_user_projects_not_finished(user):

    users_projects_not_completed = requests.get('https://api.ravelry.com/projects/' + user + '/list.json', 
                                                auth=(creds['id'], creds['key']))

    df = pd.DataFrame(users_projects_not_completed.json()['projects'])
    users_projects_not_completed = list(set(df[df['status_name'] != 'Finished']['pattern_id'].dropna()))
    return users_projects_not_completed

def get_user_queue(user):

    users_queue = requests.get('https://api.ravelry.com/people/' + user + '/queue/list.json?page_size=100', 
                                                auth=(creds['id'], creds['key']))
    
    users_queue = list(set(pd.DataFrame(users_queue.json()['queued_projects'])['pattern_id'].dropna()))

    return users_queue

def get_user_favorites(user):

    users_favourites = requests.get('https://api.ravelry.com/people/' + user + '/favorites/list.json?page_size=100', 
                                                auth=(creds['id'], creds['key']))
    
    df = pd.DataFrame(users_favourites.json()['favorites'])
    users_favourites = list(pd.DataFrame(list(df[df['type'] == 'pattern']['favorited']))['id'])
    
    return users_favourites

In [195]:
def top_rated_15(user):
    
    if user in users_list:
    
        predictions = []
        
        users_patterns = list(input_df[input_df['user'] == user]['pattern_id'])
        users_favourites = get_user_favorites(user)
        users_queue = get_user_queue(user)
        users_projects_not_completed = get_user_projects_not_finished(user)
        
        previously_interacted = users_patterns + users_favourites + users_queue + users_projects_not_completed
        
        remaining_patterns = [x for x in patterns_list if x not in previously_interacted]

    
        for pattern in remaining_patterns:
            x = algo.predict(user, pattern)
            predictions.append(x)
        
        predictions_df = pd.DataFrame({"user": [prediction.uid for prediction in predictions],
                                       "item": [prediction.iid for prediction in predictions],
                                       "actual": [prediction.r_ui for prediction in predictions],
                                       "estimated" :[prediction.est for prediction in predictions]})
    
        predictions_df = predictions_df.sort_values('estimated', ascending = False).head(15)
    
        return predictions_df
    
    else: 
        
        return 'User not in model'
        

In [196]:
top_rated_15('Ona')

,user,item,actual,estimated
5716,Ona,353871,None,3.978722
1174,Ona,3654,None,3.967730
676,Ona,202733,None,3.965192
119,Ona,8166,None,3.963573
20,Ona,443533,None,3.956684
7124,Ona,49919,None,3.948624
282,Ona,103556,None,3.948200
1874,Ona,225328,None,3.946841
542,Ona,1149,None,3.943319
2660,Ona,294119,None,3.941305


In [197]:
top_rated_15('bellybutton')

,user,item,actual,estimated
512,bellybutton,211562,None,3.843137
2789,bellybutton,3156,None,3.815066
293,bellybutton,683,None,3.811477
468,bellybutton,62838,None,3.811133
20,bellybutton,443533,None,3.810762
4607,bellybutton,455358,None,3.802176
4819,bellybutton,267564,None,3.794055
162,bellybutton,156437,None,3.788071
5772,bellybutton,510237,None,3.785139
4331,bellybutton,5220,None,3.768257
